In [60]:
import pandas as pd
import numpy as np
import sys
import os
import glob
from clawpack.geoclaw import topotools
import breach_randomization as br
import shutil

In [5]:
# Load the masked island data
masked_island = '/home/catherinej/BarrierBreach/src/barrierbreach/east_masked_moriches.npz'
with np.load(masked_island) as npz:
    island = np.ma.MaskedArray(**npz)
# iterate over all columns
import itertools
topo = topotools.read_netcdf('/home/catherinej/bathymetry/moriches.nc')
filter_region = [-72.885652,-72.634247,40.718299,40.828344]
topo_data = topo.crop(filter_region)

def locate_topo_index(south_idx, north_idx, lon_idx):
    south = topo_data.y[south_idx]
    north = topo_data.y[north_idx]
    lon = topo_data.x[lon_idx]
    return south, north, lon

gauge_locs = pd.read_csv('/home/catherinej/BarrierBreach/data/ocean_gauges.csv')
gauge_locs = gauge_locs.drop(['Unnamed: 0', 'dist'], axis=1)
island_idxs = list(zip(*np.where(island.mask==False)))
lons = np.unique([lon for lat,lon in island_idxs])
lats = np.unique([lat for lat,lon in island_idxs])
num_viable = 0
for l in lons:

    south, north, lon = locate_topo_index(lats[0], lats[-1], l)
    max_dune = max_dune_height(topo_data, lats[0], lats[-1], l)
    df = find_nearest_gauges(gauge_locs.copy(), lon, (south + north)/2, 1000)
    # print(df.index.values)
    gdata = load_gauge_data(df.index.values, '/home/catherinej/original_study_300k_r_archive/reference_gauges/')
    viable = check_location_viability(gdata, max_dune)
    if viable:
        num_viable +=1
print(num_viable)
# check to make sure its viable
# increase count by 1

306


In [84]:
import matplotlib.pyplot as plt
# plt.pcolormesh(topo_data.X, topo_data.Y, np.where(island.mask==False))
len(island_idxs)

44061

In [4]:
def max_dune_height(topo_data, ylow, yhigh, x):
    return topo_data.Z[ylow:yhigh, x].max()

def calc_distance(lat1, lat2, lon1, lon2):
    from math import sin, cos, sqrt, atan2, radians
    R = 6373.0 * 1000 # convert km to m
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2)**2 + cos(lat1)*cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance


def gauges_locs_semicircle(gauge_x, gauge_y, center_x, 
                           center_y, radius, lon1, lat1):
    v1 = [lon1 - center_x, lat1 - center_y] # vect1 = b - center_pt
    v2 = [gauge_x - center_x, gauge_y - center_y] # p - center_pt
    xp = v1[0]*v2[1] - v1[1]*v2[0]
    dist_to_center = calc_distance(gauge_y, center_y, gauge_x, center_x)
    # print('distance to center', dist_to_center)
    if xp < 0: # gauge is to right of line (ie ocean)
        if dist_to_center <= radius:
            return dist_to_center

def get_vector_points(lat, lon, bearing, distance):
    import math

    R = 6378.1 * 1000 #Radius of the Earth (m)
    brng = math.radians(bearing) #Bearing is 90 degrees converted to radians.
    d = distance #Distance in km

    #lat2  52.20444 - the lat result I'm hoping for
    #lon2  0.36056 - the long result I'm hoping for.

    lat1 = math.radians(lat) #Current lat point converted to radians
    lon1 = math.radians(lon) #Current long point converted to radians

    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) +
         math.cos(lat1)*math.sin(d/R)*math.cos(brng))

    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),
                 math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)

    return lat2, lon2

def get_bearing():
    import math
    # Upper adn lower points along moriches barrier island
    llon = -72.875
    ulon = -72.59
    llat = 40.73
    ulat = 40.81
    dLon = (ulon - llon)
    x = math.cos(math.radians(ulat)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(llat)) * math.sin(math.radians(ulat)) - \
        math.sin(math.radians(llat)) * math.cos(math.radians(ulat)) * \
        math.cos(math.radians(dLon))
    brng = np.arctan2(x,y)
    brng = np.degrees(brng)

    return brng

def find_nearest_gauges(df, breach_lon, breach_lat, distance):
    # df = pd.read_csv('/home/catherinej/BarrierBreach/src/visualization/ocean_gauges.csv')
    # df = df.drop(['Unnamed: 0', 'dist'], axis=1)
    bearing = get_bearing()
    lat2, lon2 = get_vector_points(breach_lat, breach_lon, bearing, distance)
    df['dist'] = [gauges_locs_semicircle(x, y, breach_lon, breach_lat, distance,
                                        lon2, lat2) for x, y in zip(df['lon'],
                                                                    df['lat'])]
    return df[~df['dist'].isnull()]
    # return df


def load_gauge_data(gauge_names, PATH):
    import os
    gauge_data_list = []
    for gauge in gauge_names:
        if gauge != 40:
            cols = ['Time', f'{gauge}_eta']
            gauge_path = os.path.join(PATH, f'gauge5{gauge:04}.txt')
            df = pd.read_csv(gauge_path, skiprows=4, header=None, delim_whitespace=True,
                                usecols=[1,5], index_col='Time', names=cols)
            gauge_data_list.append(df)
    if len(gauge_data_list) > 1:
        gauge_data_df = pd.concat(gauge_data_list, axis=1)
        return gauge_data_df
    else:
        return gauge_data_list
    
    
    
    
def check_location_viability(gdata, max_dune):
    """Checks to make sure that a breach location reaches the required minimum % of dune height

    Args:
        df (_dataframe_): data frame of all tide gauge timeseries
        max_dune (_type_): max dune height at chosen breach location
    """
    breach_time = []
    x_percent = max_dune * .24
    cols_greater = (gdata>= x_percent).any()
    # print('Cols greater',cols_greater)
    if cols_greater.any():
        return True
    else:
        # print('This location is not viable')
        return False

In [59]:
gdata

[]

In [52]:
import pygmt
fig = pygmt.Figure()
fig.basemap(region=filter_region, projection='M3')
fig.coast(shorelines=True)
fig.plot(x=lon, y=south, style='c0.05c', color='red')
fig.plot(x=gauge_locs['lon'].iloc[-1], y=gauge_locs['lat'].iloc[-1], style='c0.05c', color='blue')
fig.show()

basemap [ERROR]: Must specify at least one of -A, -B, -D, -L, -T


GMTCLibError: Module 'basemap' failed with status code 72:
basemap [ERROR]: Must specify at least one of -A, -B, -D, -L, -T

In [53]:
gauge_locs

,lon,lat
0,-72.583333,40.806531
1,-72.587500,40.806177
2,-72.591667,40.806547
3,-72.595000,40.805151
4,-72.597500,40.803837
...,...,...
87,-72.865833,40.728706
88,-72.869167,40.727642
89,-72.872500,40.727093
90,-72.876667,40.726799


In [54]:
bearing = get_bearing()
lat2, lon2 = get_vector_points(south, lon, bearing, 1000)
print(lat2, lon2)
gauge_locs['dist'] = [gauges_locs_semicircle(x, y, lon, (south+north)/2, 1000,
                                    lon2, lat2) for x, y in zip(gauge_locs['lon'],
                                                                gauge_locs['lat'])]
df = gauge_locs[~gauge_locs['dist'].isnull()]
df

40.728829613302 -72.62319697520226


,lon,lat,dist


In [ ]:
def find_nearest_gauges(df, breach_lon, breach_lat, distance):
    # df = pd.read_csv('/home/catherinej/BarrierBreach/src/visualization/ocean_gauges.csv')
    # df = df.drop(['Unnamed: 0', 'dist'], axis=1)
    bearing = get_bearing()
    lat2, lon2 = get_vector_points(breach_lat, breach_lon, bearing, distance)
    df['dist'] = [gauges_locs_semicircle(x, y, breach_lon, breach_lat, distance,
                                        lon2, lat2) for x, y in zip(df['lon'],
                                                                    df['lat'])]
    df = df[~df['dist'].isnull()]
    return df
